# ML Lab Assignemnt 1
- Sai Karthik 
AP21110010310
CSE-E



1. Implement Linear Regression and calculate sum of residual error on the following
Datasets.
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y = [1, 3, 2, 5, 7, 8, 8, 9, 10, 12]
#
i) Compute the regression coefficients using analytic formulation and calculate Sum
Squared Error (SSE) and R2 value.
#
ii) Implement gradient descent (both Full-batch and Stochastic with stopping
criteria) on Least Mean Square loss formulation to compute the coefficients of
regression matrix and compare the results using performance measures such as R2
SSE etc.


In [1]:
import numpy as np  
import pandas as pd

In [2]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
y = np.array([1, 3, 2, 5, 7, 8, 8, 9, 10, 12])

In [3]:
def linear_regression(x, y):
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    slope = np.sum((x - x_mean) * (y - y_mean)) / np.sum((x - x_mean) ** 2)
    intercept = y_mean - slope * x_mean
    
    y_pred = slope * x + intercept
    
    sse = np.sum((y - y_pred) ** 2)
    
    ss_total = np.sum((y - y_mean) ** 2)
    r2 = 1 - (sse / ss_total)
    
    return slope, intercept, sse, r2

slope, intercept, sse, r2 = linear_regression(x, y)

In [4]:
print("Slope:", slope)
print("Intercept:", intercept)
print("Sum of Squared Error (SSE):", sse)
print("R^2 value:", r2)

Slope: 1.1696969696969697
Intercept: 1.2363636363636363
Sum of Squared Error (SSE): 5.624242424242423
R^2 value: 0.952538038613988


In [5]:
def linear_regression_gradient_descent(x, y, learning_rate=0.01, epochs=1000, batch_size=None):
    n = len(x)
    slope = 0
    intercept = 0
    
    if batch_size is None:
        batch_size = n
    
    for epoch in range(epochs):
        if batch_size == n:
            x_batch = x
            y_batch = y
        else:
            indices = np.random.choice(n, batch_size)
            x_batch = x[indices]
            y_batch = y[indices]
        
        y_pred = slope * x_batch + intercept
        
        slope_gradient = -(2 / batch_size) * np.sum(x_batch * (y_batch - y_pred))
        intercept_gradient = -(2 / batch_size) * np.sum(y_batch - y_pred)
        
        slope -= learning_rate * slope_gradient
        intercept -= learning_rate * intercept_gradient
        
    y_pred = slope * x + intercept
    
    sse = np.sum((y - y_pred) ** 2)
    
    y_mean = np.mean(y)
    ss_total = np.sum((y - y_mean) ** 2)
    r2 = 1 - (sse / ss_total)
    
    return slope, intercept, sse, r2



In [10]:
slope_batch, intercept_batch, sse_batch, r2_batch = linear_regression_gradient_descent(x, y)
print("\nFull-batch gradient descent:")
print("Slope:", slope_batch)
print("Intercept:", intercept_batch)
print("Sum of Squared Error (SSE):", sse_batch)
print("R2 value:", r2_batch)

slope_stochastic, intercept_stochastic, sse_stochastic, r2_stochastic = linear_regression_gradient_descent(x, y, batch_size=2)
print("\nStochastic gradient descent:")
print("Slope:", slope_stochastic)
print("Intercept:", intercept_stochastic)
print("Sum of Squared Error (SSE):", sse_stochastic)
print("R2 value:", r2_stochastic)



Full-batch gradient descent:
Slope: 1.170263693076768
Intercept: 1.2328099487610318
Sum of Squared Error (SSE): 5.624278989977716
R2 value: 0.9525377300423822

Stochastic gradient descent:
Slope: 1.1684728386473255
Intercept: 1.275337914338942
Sum of Squared Error (SSE): 5.63556557313237
R2 value: 0.9524424846149168


2. Download Boston Housing Rate Dataset. Analyse the input attributes and find out the
attribute that best follow the linear relationship with the output price. Implement both the
analytic formulation and gradient descent (Full-batch, stochastic) on LMS loss
formulation to compute the coefficients of regression matrix and compare the results.S

In [13]:
df=pd.read_csv("BostonHousing.csv")

In [14]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
